# 📊 Tesla & GameStop Gelir Analizi
Bu notebook, yfinance ve web scraping kullanarak Tesla ve GameStop şirketlerinin hisse ve gelir verilerini analiz eder.

In [ ]:
import yfinance as yf

# Tesla'nın hisse sembolü
tesla = yf.Ticker("TSLA")

# Verileri en uzun periyot için alıyoruz
tesla_data = tesla.history(period="max")

# İlk 5 satırı görelim
tesla_data.reset_index(inplace=True)
tesla_data.head()

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

company = "TSLA"  # "GME" yazarsan GameStop verisi gelir
url = f"https://www.macrotrends.net/stocks/charts/{company}/{company.lower()}/revenue"

headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

tables = soup.find_all("table")
revenue_table = tables[1]
df_revenue = pd.read_html(str(revenue_table))[0]
df_revenue.columns = ["Date", "Revenue"]
df_revenue.head()

In [ ]:
# GameStop verisi
gamestop = yf.Ticker("GME")
gamestop_data = gamestop.history(period="max")
gamestop_data.reset_index(inplace=True)
gamestop_data.head()

In [ ]:
company = "GME"
url = f"https://www.macrotrends.net/stocks/charts/{company}/{company.lower()}/revenue"

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

tables = soup.find_all("table")
revenue_table = tables[1]
df_gme_revenue = pd.read_html(str(revenue_table))[0]
df_gme_revenue.columns = ["Date", "Revenue"]
df_gme_revenue.head()

In [ ]:
df_revenue["Date"] = pd.to_datetime(df_revenue["Date"])
df_revenue["Year"] = df_revenue["Date"].dt.year
tesla_data["Year"] = tesla_data["Date"].dt.year

tesla_avg_price = tesla_data.groupby("Year")["Close"].mean().reset_index()
tesla_revenue_by_year = df_revenue.groupby("Year")["Revenue"].first().reset_index()

tesla_indicator = pd.merge(tesla_avg_price, tesla_revenue_by_year, on="Year", how="inner")
tesla_indicator.head()

In [ ]:
df_gme_revenue["Date"] = pd.to_datetime(df_gme_revenue["Date"])
df_gme_revenue["Year"] = df_gme_revenue["Date"].dt.year
gamestop_data["Year"] = gamestop_data["Date"].dt.year

gme_avg_price = gamestop_data.groupby("Year")["Close"].mean().reset_index()
gme_revenue_by_year = df_gme_revenue.groupby("Year")["Revenue"].first().reset_index()

gme_indicator = pd.merge(gme_avg_price, gme_revenue_by_year, on="Year", how="inner")
gme_indicator.head()